In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [15]:
# Import required libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import unicodedata
from datetime import datetime

#### Incremental Ingestion

In [85]:
#Get DB credentials
from dotenv import load_dotenv
from pathlib import Path
import os

dotenv_path = Path('db_credentials.env')
load_dotenv(dotenv_path=dotenv_path)

DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
ETL_DATE = os.getenv('ETL_DATE')

In [80]:
import psycopg2

# Connect to the PostgreSQL database
connection = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)

cursor = connection.cursor()

try:
    cursor.execute('SELECT MAX("Review Date") FROM silver_airline_quality_reviews;')
    
    max_date = cursor.fetchone()[0]
    
    print("The maximum date is:", max_date)

except (Exception, psycopg2.DatabaseError) as error:
    print(f"Error: {error}")
    
finally:
    cursor.close()
    connection.close()

The maximum date is: 2025-03-22 00:00:00


In [87]:
# For Incremental Ingestion (enabled if ETL_DATE set to 'CURRENT_DATE')
GET_REVIEWS_FROM_DATE = pd.to_datetime(max_date)

if GET_REVIEWS_FROM_DATE is None:
    GET_REVIEWS_FROM_DATE = pd.to_datetime('2002-01-01')
    
if ETL_DATE == 'CURRENT_DATE':
    ETL_DATE = datetime.today().strftime('%Y%m%d')

print(GET_REVIEWS_FROM_DATE)
print(ETL_DATE)

2025-03-22 00:00:00
20250328


#### Airline Quality Reviews Web Scraping

In [19]:
# Initialize an empty list to store the airline names
airline_names = []

# Set custom User-Agent in the headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36"
}

# Loop through each letter of the alphabet (from A to Z)
for letter in range(ord('A'), ord('Z') + 1):
    l = chr(letter)  # Get the character corresponding to the current ASCII code (e.g., 'A' for 65, 'B' for 66, etc.)

    # Build the URL for the specific letter to fetch airline names starting with that letter
    url = f"https://www.airlinequality.com/review-pages/a-z-airline-reviews/#a2z-ldr-{l}"

    # Send an HTTP GET request to fetch the webpage's HTML content using the custom User-Agent in the headers
    html_text = requests.get(url, headers=headers).text

    # Parse the HTML content using BeautifulSoup with the "lxml" parser
    soup = BeautifulSoup(html_text, "lxml")

    # Create a unique identifier (URI) to locate the container for the current letter
    uri = f"a2z-ldr-{l}"

    # Find the container that holds the list of airlines starting with the current letter
    container = soup.find("div", {"id": uri})

    # Loop through each list item in the container and extract the airline name
    for items in container.find_all("li"):
        airline_names.append(items.text)

In [20]:
#airline_names

In [21]:
# to convert names of airlines into url
start = "https://www.airlinequality.com/airline-reviews/"
end = "?sortby=post_date%3ADesc&pagesize=100"
airline_url = []
for items in airline_names:
    #converting names to lower case
    items = items.lower()
    # Replace special characters with their ASCII equivalents
    items = unicodedata.normalize('NFKD', items).encode('ASCII', 'ignore').decode('utf-8')
    # Replacing space with hypen
    items = items.replace(" ", "-")
    #creating URL
    airline_url.append(start + items + "/"+end)

In [22]:
#airline_url

In [23]:
df_airline = pd.DataFrame({"Name":airline_names,"Links":airline_url})

#### Option to Get for a specific airline only

In [25]:
df_airline_filtered = df_airline
#df_airline_filtered = df_airline[df_airline['Name'].str.contains('West Jet')]

df_airline_filtered

,Name,Links
0,AB Aviation,https://www.airlinequality.com/airline-reviews...
1,Adria Airways,https://www.airlinequality.com/airline-reviews...
2,Aegean Airlines,https://www.airlinequality.com/airline-reviews...
3,Aer Lingus,https://www.airlinequality.com/airline-reviews...
4,Aero VIP,https://www.airlinequality.com/airline-reviews...
...,...,...
567,Yangon Airways,https://www.airlinequality.com/airline-reviews...
568,Yemenia,https://www.airlinequality.com/airline-reviews...
569,Yeti Airlines,https://www.airlinequality.com/airline-reviews...
570,Zambia Airways,https://www.airlinequality.com/airline-reviews...


#### Add review image urls

In [27]:
columns = ["Aircraft","Type Of Traveller","Seat Type","Route","Date Flown","Seat Comfort","Cabin Staff Service","Food & Beverages",\
           "Ground Service","Inflight Entertainment","Wifi & Connectivity","Value For Money","Recommended","Top Review Image Url"]

df_columns = ["Airline Name","Overall_Rating","Review_Title","Review Date","Verified","Review","Top Review Image Url","Aircraft",\
              "Type Of Traveller","Seat Type","Route","Date Flown","Seat Comfort","Cabin Staff Service","Food & Beverages","Ground Service",\
              "Inflight Entertainment","Wifi & Connectivity","Value For Money","Recommended"]

In [28]:
reviews = []

# Loop through each row in the 'df_airline' DataFrame
for index, row in df_airline_filtered.iterrows():
    # Send an HTTP GET request to fetch the HTML content of the airline's review page
    html = requests.get(row['Links'], headers=headers).text
    bs = BeautifulSoup(html, "html.parser")

    # Find the container that holds the airline's review information
    container = bs.find("article", {"class": "comp comp_reviews-airline querylist position-content"})

    # Check if the container exists (i.e., the review page is valid)
    if container:
        # Extract the airline's name from the 'Name' column of the DataFrame
        print(row['Name'])

        # Loop through each review article in the container and extract relevant details
        for items in container.find_all("article"):
            verified = False
            # Extract the overall rating from the review
            rating = items.find("div", {"class": "rating-10"})
            #if rating:
            #    rating = rating.text.strip()[:1]
            #else:
            #    rating = None
            if rating and rating.text.strip()[:2] == '10':
                rating = '10'
            else:
                rating = rating.text.strip()[:1] if rating and rating.text else None

            # Extract the review title from the review
            title = items.find("h2")
            if title:
                title = title.text
            else:
                title = None

            # Extract the review date from the review
            time = items.find("h3").find("time")
            if time:
                time = time.text
            else:
                time = None

            if time is None or pd.to_datetime(time) <= GET_REVIEWS_FROM_DATE:
                #print(f'{time}: Skipping to next review')
                continue
                
            # Extract the review content from the review
            text = items.find("div", {"class": "text_content"}).text
            text = text.split("|")
            if len(text) == 1:
                review = text[0]
            else:
                if text[0] == '✅ Trip Verified ':
                    verified = True
                review = text[1]

            # Extract the review image url from the review
            first_div = items.find("div", {"class": "large_review_img"})
            if first_div and first_div.img:
                first_image_src = first_div.img['src']
                #print(first_image_src)
            else:
                #print("No image found.")
                first_image_src = None
                
            # Extract ratings for specific categories from the review
            table = items.find("table")
            tab = [None] * 13
            for item in table.find_all("tr"):
                i = 0
                for td in item.find_all("td"):
                    if i == 0:
                        condition = td.text
                        # Finding index of the given condition in the 'columns' list
                        ind = columns.index(condition)
                        i = 1
                    else:
                        # Checking if it's a rating or not
                        if td.find("span") is None:
                            value = td.text
                            tab[ind] = value
                        # In case it's a rating, counting stars filled (i.e., stars given)
                        else:
                            value = 0
                            for star in td.find_all("span", {"class": "star fill"}):
                                value += 1
                            tab[ind] = value

            # Store the extracted review data in a list named 'data'
            data = [row['Name'], rating, title, time, verified, review, first_image_src] + tab
            reviews.append(data)

# The 'reviews' list now contains all the extracted review data for different airlines.

AB Aviation
11th November 2019: Skipping to next review
25th June 2019: Skipping to next review
25th June 2019: Skipping to next review
Adria Airways
28th September 2019: Skipping to next review
24th September 2019: Skipping to next review
17th September 2019: Skipping to next review
6th September 2019: Skipping to next review
24th August 2019: Skipping to next review
6th August 2019: Skipping to next review
12th October 2018: Skipping to next review
5th October 2018: Skipping to next review
29th July 2018: Skipping to next review
19th July 2018: Skipping to next review
30th June 2018: Skipping to next review
24th June 2018: Skipping to next review
4th May 2018: Skipping to next review
11th March 2018: Skipping to next review
5th December 2017: Skipping to next review
20th November 2017: Skipping to next review
27th October 2017: Skipping to next review
16th September 2017: Skipping to next review
19th April 2017: Skipping to next review
27th January 2017: Skipping to next review
10th 

In [29]:
#reviews

In [41]:
#creating reviews dataframe
df = pd.DataFrame(reviews, columns=df_columns)

In [43]:
df.shape

(52, 20)

In [45]:
df.sample(10)

,Airline Name,Overall_Rating,Review_Title,Review Date,Verified,Review,Top Review Image Url,Aircraft,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Inflight Entertainment,Wifi & Connectivity,Value For Money,Recommended
16,Emirates,1,“The most unprofessional staff”,25th March 2025,True,The most unprofessional staff I’ve ever seen....,None,Boeing 777,Couple Leisure,Economy Class,Phuket to Dubai,March 2025,1.0,1.0,1.0,2.0,3.0,1.0,1,no
15,Egyptair,1,“they made no effort to help”,26th March 2025,False,Avoid EgyptAir – A Disappointing Experience. ...,None,None,Solo Leisure,Economy Class,Mumbai to Dublin via Egypt,March 2025,1.0,2.0,2.0,1.0,2.0,2.0,2,no
13,easyJet,1,“Their entire conduct was unnecessary”,25th March 2025,True,I was stopped to check the size of my bag. I ...,https://www.airlinequality.com/wp-content/uplo...,None,Couple Leisure,Economy Class,Keflavik to Manchester,March 2025,4.0,5.0,5.0,1.0,2.0,1.0,2,no
39,Super Air Jet,1,“very bad ticketing management”,28th March 2025,False,Have a very bad ticketing management. Super A...,None,A320,Family Leisure,Economy Class,Jakarta to Denpasar,March 2025,1.0,1.0,1.0,1.0,1.0,1.0,1,no
25,Frontier Airlines,1,“Choose anyone but Frontier”,24th March 2025,True,"It's been many, many years since I've been en...",None,None,Couple Leisure,Economy Class,Tampa to Boston,March 2025,1.0,2.0,NaN,1.0,NaN,NaN,1,no
20,Fiji Airways,8,"""attentiveness and kindness of the crew""",28th March 2025,True,"Returning from my honeymoon to Mexico, we f...",https://www.airlinequality.com/wp-content/uplo...,A350,Couple Leisure,Business Class,Los Angeles to Nadi,March 2025,5.0,5.0,3.0,5.0,3.0,4.0,5,yes
4,AirAsia X,2,"""flight is delay for 20 hours""",24th March 2025,True,"Worse experience, my flight suppose to depa...",None,None,Business,Economy Class,Amritsar to Perth via Kuala Lumpur,March 2025,1.0,1.0,1.0,1.0,NaN,NaN,1,no
32,Qatar Airways,2,“still not heard from them for 5 weeks”,27th March 2025,False,My flight from Perth to Heathrow was cancelle...,None,Boeing 777,Solo Leisure,Economy Class,Perth to London Heathrow via Doha,July 2024,1.0,2.0,2.0,3.0,3.0,NaN,3,no
22,flydubai,3,"""demanded a fee for my laptop bag""",24th March 2025,True,During my transit in Dubai on a Riyadh to Mu...,None,None,Solo Leisure,Economy Class,Riyadh to Multan via Dubai,March 2025,2.0,4.0,4.0,2.0,3.0,NaN,3,no
46,VivaAerobús,1,"""I do not recommend this airline""",24th March 2025,True,I wanted to travel with my family and servi...,None,None,Family Leisure,Economy Class,Bacalar to Mexico City,March 2025,2.0,1.0,1.0,1.0,1.0,1.0,1,no


In [47]:
df.to_csv(f"data/bronze_{ETL_DATE}_Airline_Reviews_withImageUrls.csv")

In [49]:
df[df['Overall_Rating'] == '10'].head()

,Airline Name,Overall_Rating,Review_Title,Review Date,Verified,Review,Top Review Image Url,Aircraft,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Inflight Entertainment,Wifi & Connectivity,Value For Money,Recommended
1,Air France,10,"""excellent customer service""",28th March 2025,True,My son was travelling on Kids Solo as an un...,None,None,Family Leisure,Economy Class,Dublin to Singapore,March 2025,NaN,NaN,NaN,5.0,NaN,NaN,5,yes
10,Cyprus Airways,10,“wonderful staff”,26th March 2025,False,I am beyond happy with the assistance I recei...,None,None,Solo Leisure,Economy Class,Dubai to Larnaca,March 2025,5.0,5.0,NaN,5.0,5.0,NaN,5,yes
29,Porter Airlines,10,“Thank you Erin and Porter”,24th March 2025,False,I have flown Porter on other occasions and ha...,None,None,Solo Leisure,Economy Class,St. John's to Halifax,March 2025,3.0,5.0,4.0,4.0,NaN,NaN,5,yes
47,Volotea,10,"""The service is up to par""",24th March 2025,True,A very decent company. The service is up to...,None,None,Couple Leisure,Economy Class,Barcelone to Lille,March 2025,4.0,5.0,NaN,5.0,3.0,2.0,5,yes
48,Volotea,10,“a fantastic airline”,22nd March 2025,False,"Es una aerolínea fantástica, el personal es m...",None,None,Solo Leisure,Economy Class,Bilbao to Gran Canaria,March 2025,5.0,5.0,5.0,5.0,5.0,4.0,5,yes
